In [ ]:
cd ../input/dataclass06/dataclasses-0.6

In [ ]:
!pip install .

In [ ]:
cd ../../../working

In [ ]:
!pip3 install ../input/torch170/torch-1.7.0-cp37-cp37m-manylinux1_x86_64.whl

In [ ]:
!mkdir -p /root/.cache/torch/hub/checkpoints

In [ ]:
!cp ../input/pretrained/for_kaggle/*.pt* /root/.cache/torch/hub/checkpoints

In [ ]:
!cp -r ../input/pretrained/for_kaggle/facebookresearch_WSL-Images_master /root/.cache/torch/hub/

In [ ]:
!cp -r ../input/pretrained/for_kaggle/intel-isl_MiDaS_master /root/.cache/torch/hub/

In [ ]:
cd ../input/pretrained/for_kaggle/EfficientNet-PyTorch-master

In [ ]:
!pip install .

In [ ]:
cd ../../../../working

In [ ]:
!cp -r ../input/image-models/pytorch-image-models-master/* .

In [ ]:
from efficientnet_pytorch import EfficientNet 
from torchvision import models
import torch 
import cv2 
import glob 
import torch.nn.functional as F
import timm
import numpy as np

midas = torch.hub.load("intel-isl/MiDaS", "MiDaS")
midas.cuda().eval()
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
transform = midas_transforms.default_transform

efficient = EfficientNet.from_name('efficientnet-b5', num_classes=5)
efficient.load_state_dict(torch.load('../input/ensemblev5/eff_best.pth'))
efficient.eval().cuda()

seresnext = timm.create_model('seresnext101_32x4d', num_classes=5)
seresnext.load_state_dict(torch.load('../input/ensemblev5/seresnext_best.pth'))
seresnext.eval().cuda()

print('Models have been loaded...\n')

def processor(image):
    img = cv2.resize(image, (512,512))
    img = (img - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]
    image = torch.tensor(img.transpose(2,0,1), dtype=torch.float).unsqueeze(0).cuda()
    return image

def get_depth(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    input_batch = transform(img).to("cuda")
    with torch.no_grad():
        prediction = midas(input_batch)

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode="bicubic",
        ).squeeze()
    output = prediction.cpu().numpy()
    img_min = np.min(output)
    img_max = np.max(output)
    return output > ((img_min+img_max)/3)

def crop_image(image, depth):
    depth = depth.astype(int)
    mask_3d = np.stack((depth,depth,depth),axis=2)
    masked_arr = np.where(mask_3d==1,img,mask_3d).astype(np.uint8)
    c = np.where(masked_arr!=[0,0,0])
    x_max = max(c[1])
    x_min = min(c[1])
    y_max = max(c[0])
    y_min = min(c[0])
    image = masked_arr[y_min:y_max, x_min:x_max]
    return image

files = glob.glob('../input/cassava-leaf-disease-classification/test_images/*')
names, labels = [], []

#images = ['1000015157.jpg','1000201771.jpg','100042118.jpg','1000723321.jpg','1000812911.jpg','1000837476.jpg','1000910826.jpg','1001320321.jpg','1001723730.jpg']



for file in files:
    img = cv2.imread(file)
    process = img 
    depth = get_depth(process)
    img = crop_image(img, depth)
    img = processor(img)
    
    se_out = seresnext(img)
    eff_out = efficient(img)
    total = (se_out + eff_out)/2
    names.append(file.split('/')[-1])
    labels.append(int(torch.argmax(torch.nn.functional.softmax(total)).detach().cpu()))
    
import pandas as pd

dataset = {'image_id': names,
        'label': labels
        }

df = pd.DataFrame(dataset, columns= ['image_id', 'label'])

df.to_csv ('submission.csv', index = False, header=True)